In [2]:
import pandas as pd
import os

In [3]:
#2014
    
ben_api = pd.read_csv("ben_api_2014.csv")
mal_api = pd.read_csv("mal_api_2014.csv")

In [4]:
ben_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39930 entries, 0 to 39929
Columns: 1849 entries, apkname to Lorg/w3c/dom/Element;.getElementsByTagNameNS:(Ljava/lang/String;Ljava/lang/String;)Lorg/w3c/dom/NodeList;
dtypes: int64(1848), object(1)
memory usage: 563.3+ MB


In [5]:
mal_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10073 entries, 0 to 10072
Columns: 1849 entries, apkname to Lorg/w3c/dom/Element;.getElementsByTagNameNS:(Ljava/lang/String;Ljava/lang/String;)Lorg/w3c/dom/NodeList;
dtypes: int64(1848), object(1)
memory usage: 142.1+ MB


In [6]:
#ben, mal 수 맞춰주기

ben_api_del = ben_api[:10073]

In [7]:
import numpy as np

ben = np.zeros((len(ben_api_del), 1), int)

mal = np.ones((len(mal_api), 1), int)

label = np.concatenate((ben, mal), axis=0)
label

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [8]:
len(label)

20146

In [9]:
api2014 = pd.concat([ben_api_del, mal_api])

In [10]:
api2014.drop('apkname', axis=1, inplace=True)

In [11]:
len(ben_api_del), len(mal_api)

(10073, 10073)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(api2014, label, test_size=0.2, random_state=42)

In [13]:
x_train.shape

(16116, 1848)

In [14]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

LinearRegression()

In [15]:
lin_reg.predict(x_test)

array([[0.77991978],
       [1.20274722],
       [0.03919895],
       ...,
       [0.41671476],
       [1.25006236],
       [0.90457706]])

In [16]:
from sklearn.metrics import mean_squared_error

predictions2014 = lin_reg.predict(x_train)
lin_mse = mean_squared_error(y_train, predictions2014)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.28032641540632475

In [17]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(y_train, predictions2014)
lin_mae

0.22402913657433393

In [18]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lin_reg, x_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [19]:
def display_scores(scores):
    print("점수:", scores)
    print("평균:", scores.mean())
    print("표준 편차:", scores.std())

display_scores(tree_rmse_scores)

점수: [0.35517846 0.42374419 0.31896243 0.42901583 0.41705934 0.35639016
 0.37779541 0.43674377 0.310966   0.38054229]
평균: 0.38063978786000285
표준 편차: 0.043098989380614725


In [20]:
#트리 모델 별 성능 평가

#결정 트리 분류기
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time

start_time = time.time()

tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(x_train, y_train)
end_time = time.time() - start_time

tree_trainpred = tree_clf.predict(x_train) 
tree_trainacc = accuracy_score(y_train, tree_trainpred)

tree_testpred = tree_clf.predict(x_test)
tree_testacc = accuracy_score(y_test, tree_testpred)

print(f"모델 정확도(train) : {tree_trainacc :.4f}")
print(f"모델 정확도(test) : {tree_testacc :.4f}")
print(f"학습 시간 : {end_time :.4f}")

모델 정확도(train) : 0.9958
모델 정확도(test) : 0.9772
학습 시간 : 0.7671


In [21]:
## GridSearchCV 이용해서 교차검증&최적의 파라미터 찾기
from sklearn.model_selection import GridSearchCV

params = {
    'min_samples_split':[11, 12, 13],
    'max_depth': [13, 14]
}

grid_cv = GridSearchCV(tree_clf, param_grid=params, cv=5, verbose=1)
grid_cv.fit(x_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [13, 14],
                         'min_samples_split': [11, 12, 13]},
             verbose=1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


# 최적값을 구하고 싶은 파라미터를 정리 
dists = {
    'max_depth' : [3,5,10,15],
    'max_features' : [3,5,10],
    'n_estimators' : [80, 100, 150, 200]
}

# RandomizedSearchCV 작성
rs_cv = RandomizedSearchCV(
    tree_clf,
    param_distributions=dists, # 파라미터 입력
    n_iter = 500,   # random search 탐색 횟수
    cv = 5,        # cv 검증을 위한 분할 검증 횟수
    scoring='accuracy',  # 오차 평가방법
    verbose=1,     # 진행상황
    random_state = 42
  )

rs_cv.fit(x_train, y_train)

In [22]:
print("최적의 파라미터:", grid_cv.best_params_)
print("최고의 정확도 :", grid_cv.best_score_)

tree_b = grid_cv.best_estimator_
tree_b.fit(x_train, y_train)
tree_bp = tree_b.predict(x_test)
tree_bs = accuracy_score(y_test, tree_bp)
print("테스트 정확도:", tree_bs)

최적의 파라미터: {'max_depth': 14, 'min_samples_split': 12}
최고의 정확도 : 0.9777241199843865
테스트 정확도: 0.9776674937965261


In [42]:
#랜덤포레스트 분류기

from sklearn.ensemble import RandomForestClassifier

start_time = time.time()

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(x_train, y_train)
end_time = time.time() - start_time

forest_trainpred = forest_clf.predict(x_train) 
forest_trainacc = accuracy_score(y_train, forest_trainpred)

forest_testpred = forest_clf.predict(x_test)
forest_testacc = accuracy_score(y_test, forest_testpred)

print(f"모델 정확도(train) : {forest_trainacc :.4f}")
print(f"모델 정확도(test) : {forest_testacc :.4f}")
print(f"학습 시간 : {end_time :.4f}")

/tmp/ipykernel_52/1296166286.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_clf.fit(x_train, y_train)


모델 정확도(train) : 0.9958
모델 정확도(test) : 0.9789
학습 시간 : 4.8443


In [46]:
import numpy as np

importances = forest_clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest_clf.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

print("Feature Rank:")

for f in range(api2014.shape[1]):
    print("{}. {} ({:.3f})".format(f + 1, api2014.columns[indices][f], importances[indices[f]]))

Feature Rank:
1. Landroid/content/ContentResolver;.query:(Landroid/net/Uri;[Ljava/lang/String;Ljava/lang/String;[Ljava/lang/String;Ljava/lang/String;)Landroid/database/Cursor; (0.103)
2. Ljava/util/Timer;.schedule:(Ljava/util/TimerTask;JJ)V (0.041)
3. Landroid/webkit/WebView;.loadDataWithBaseURL:(Ljava/lang/String;Ljava/lang/String;Ljava/lang/String;Ljava/lang/String;Ljava/lang/String;)V (0.036)
4. Landroid/location/LocationManager;.requestLocationUpdates:(Ljava/lang/String;JFLandroid/location/LocationListener;)V (0.024)
5. Landroid/content/pm/PackageManager;.getInstalledPackages:(I)Ljava/util/List; (0.016)
6. Landroid/location/LocationManager;.requestLocationUpdates:(Ljava/lang/String;JFLandroid/location/LocationListener;Landroid/os/Looper;)V (0.015)
7. Ljava/lang/StringBuilder;.append:(Ljava/lang/Object;)Ljava/lang/StringBuilder; (0.013)
8. Landroid/net/NetworkInfo;.getTypeName:()Ljava/lang/String; (0.012)
9. Landroid/telephony/TelephonyManager;.getDeviceId:()Ljava/lang/String; (0.01

1218. Ljava/net/MulticastSocket;.wait:(JI)V (0.000)
1219. Ljava/net/NetworkInterface;.equals:(Ljava/lang/Object;)Z (0.000)
1220. Ljava/net/NetworkInterface;.getByIndex:(I)Ljava/net/NetworkInterface; (0.000)
1221. Ljava/net/NetworkInterface;.getClass:()Ljava/lang/Class; (0.000)
1222. Ljava/net/NetworkInterface;.getIndex:()I (0.000)
1223. Landroid/location/LocationManager;.addNmeaListener:(Landroid/location/OnNmeaMessageListener;Landroid/os/Handler;)Z (0.000)
1224. Landroid/location/LocationManager;.addNmeaListener:(Landroid/location/OnNmeaMessageListener;)Z (0.000)
1225. Ljava/net/NetworkInterface;.getMTU:()I (0.000)
1226. Ljava/net/NetworkInterface;.getParent:()Ljava/net/NetworkInterface; (0.000)
1227. Ljava/net/NetworkInterface;.getSubInterfaces:()Ljava/util/Enumeration; (0.000)
1228. Ljava/net/MulticastSocket;.getTTL:()B (0.000)
1229. Ljava/net/MulticastSocket;.getSoTimeout:()I (0.000)
1230. Ljava/net/MulticastSocket;.getSendBufferSize:()I (0.000)
1231. Landroid/media/AudioRecord$Bui

In [24]:
params = {
    'n_estimators': [36, 37, 38],
    'max_depth': [21, 22],
    'min_samples_leaf': [2],
    'min_samples_split': [2]
}

grid_cv = GridSearchCV(forest_clf, param_grid=params, cv=5, verbose=1)
grid_cv.fit(x_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataCon

/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [21, 22], 'min_samples_leaf': [2],
                         'min_samples_split': [2],
                         'n_estimators': [36, 37, 38]},
             verbose=1)

In [25]:
print("최적의 파라미터:", grid_cv.best_params_)
print("최고의 정확도 :", grid_cv.best_score_)

forest_b = grid_cv.best_estimator_
forest_b.fit(x_train, y_train)
forest_bp = forest_b.predict(x_test)
forest_bs = accuracy_score(y_test, forest_bp)
print("테스트 정확도:", forest_bs)

최적의 파라미터: {'max_depth': 21, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 38}
최고의 정확도 : 0.9728844864262678


/tmp/ipykernel_52/29744953.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_b.fit(x_train, y_train)


테스트 정확도: 0.9761786600496278


In [26]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=42)
gb_clf.fit(x_train, y_train)
end_time = time.time() - start_time

gb_trainpred = gb_clf.predict(x_train) 
gb_trainacc = accuracy_score(y_train, gb_trainpred)

gb_testpred = gb_clf.predict(x_test)
gb_testacc = accuracy_score(y_test, gb_testpred)

print(f"모델 정확도(train) : {gb_trainacc :.4f}")
print(f"모델 정확도(test) : {gb_testacc :.4f}")
print(f"학습 시간 : {end_time :.4f}")

/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


모델 정확도(train) : 0.9842
모델 정확도(test) : 0.9809
학습 시간 : 23.3293


In [39]:
params = {
    'n_estimators': [70, 72, 74],
    'learning_rate': [0.3, 0.35, 0.4]
}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=5, verbose=1)
grid_cv.fit(x_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             param_grid={'learning_rate': [0.3, 0.35, 0.4],
                         'n_estimators': [70, 72, 74]},
             verbose=1)

In [41]:
print("최적의 파라미터:", grid_cv.best_params_)
print("최고의 정확도 :", grid_cv.best_score_)

gb_b = grid_cv.best_estimator_
gb_b.fit(x_train, y_train)
gb_bp = gb_b.predict(x_test)
gb_bs = accuracy_score(y_test, gb_bp)
print("테스트 정확도:", gb_bs)

최적의 파라미터: {'learning_rate': 0.3, 'n_estimators': 74}
최고의 정확도 : 0.9842395768934356


/home/sermadl/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


테스트 정확도: 0.9826302729528535
